In [133]:
import pandas as pd
import seaborn as sns
import json
from bs4 import BeautifulSoup
import re
from math import ceil
%matplotlib inline
sns.set()
import shutil

In [2]:
df = pd.read_csv('./dirty_data/free_company_dataset.csv', error_bad_lines=False)

b'Skipping line 69743: expected 10 fields, saw 11\nSkipping line 101632: expected 10 fields, saw 11\n'
b'Skipping line 157754: expected 10 fields, saw 11\nSkipping line 189017: expected 10 fields, saw 11\n'
b'Skipping line 308592: expected 10 fields, saw 11\nSkipping line 312800: expected 10 fields, saw 11\n'
b'Skipping line 333445: expected 10 fields, saw 11\nSkipping line 353899: expected 10 fields, saw 11\nSkipping line 361250: expected 10 fields, saw 11\nSkipping line 374180: expected 10 fields, saw 11\nSkipping line 374658: expected 10 fields, saw 11\n'
b'Skipping line 416636: expected 10 fields, saw 11\nSkipping line 435987: expected 10 fields, saw 11\nSkipping line 455847: expected 10 fields, saw 11\n'
b'Skipping line 463320: expected 10 fields, saw 11\nSkipping line 481278: expected 10 fields, saw 11\nSkipping line 500441: expected 10 fields, saw 11\nSkipping line 504289: expected 10 fields, saw 11\n'
b'Skipping line 530687: expected 10 fields, saw 11\nSkipping line 542886: exp

In [3]:
df.head()

,country,founded,id,industry,linkedin_url,locality,name,region,size,website
0,united kingdom,2017.0,codingcodex,education management,linkedin.com/company/codingcodex,nottingham,codex,nottingham,11-50,codingwithcodex.co.uk
1,italy,2016.0,rid968fm,broadcast media,linkedin.com/company/rid968fm,rome,rid 96.8 fm,lazio,1-10,rid968.com
2,costa rica,NaN,eclipse-in-action,consumer services,linkedin.com/company/eclipse-in-action,san josé,eclipse in action,san jose,201-500,supportseven.com
3,turkey,2008.0,pergel-digital-media,internet,linkedin.com/company/pergel-digital-media,NaN,pergel digital media,NaN,1-10,pergeldigital.com
4,united states,2007.0,cougar-electric-llc,construction,linkedin.com/company/cougar-electric-llc,swartz creek,cougar electric llc,michigan,1-10,power-now.biz


In [4]:
df.country.nunique(), df.locality.nunique(), df.region.nunique()

(249, 90979, 4443)

In [5]:
df.isna().sum()/df.shape[0]*100

country         16.297923
founded         50.180549
id               0.000010
industry         9.042004
linkedin_url     0.000000
locality        33.002053
name             0.748760
region          28.927910
size             0.000000
website         24.894256
dtype: float64

In [6]:
#print(df.country.nunique())
country_df = df.groupby('country').count().sort_values('id', ascending=False)[['id', 'website']]
country_df.columns = ['n_companies', 'n_websites']
print(country_df.shape[0])
country_df.head(10)

249


,n_companies,n_websites
country,,
united states,3213829,2396446
united kingdom,935786,635134
india,396714,288193
brazil,308647,249234
france,278142,173166
canada,267602,228169
australia,262869,233152
netherlands,247567,214751
germany,234000,210501


In [7]:
industry_df = df.groupby('industry').count().sort_values('id', ascending=False)[['id', 'website']]
industry_df.columns = ['n_companies', 'n_websites']
print(industry_df.shape[0])
industry_df.head(20)

147


,n_companies,n_websites
industry,,
information technology and services,534903,451873
construction,427072,305335
marketing and advertising,385629,335856
real estate,274137,204571
management consulting,242218,194122
computer software,225451,179958
financial services,214856,167027
retail,201062,146664
"health, wellness and fitness",196334,163582


In [129]:
industry_df.tail()

NameError: name 'industry_df' is not defined

In [9]:
df['english'] = df.country.map(lambda x: x in ['united states', 'united kingdom', 'australia', 'canada'])
clean_df = df[(~df.website.isna()) & df.english & (~df.industry.isna())& (~df.locality.isna())& (~df.region.isna())]
clean_df.to_csv('./dirty_data/company_english.csv')
clean_df.shape

(2802311, 11)

In [ ]:
targets = clean_df[['website']]
CHUNK_SIZE = 200000
n_chunks = ceil(len(targets)/CHUNK_SIZE)
for i in range(n_chunks):
    targets.iloc[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE].to_csv('./targets/{}.csv'.format(i), index=False)

In [3]:
import os

def deformat_link(s):
    s = s.split('/')[2]
    if s.startswith('www'):
        s = s[4:]
    return s

os.makedirs('remaining_targets', exist_ok=True)
os.makedirs('clean_data', exist_ok=True)
os.makedirs('failed_targets', exist_ok=True)
clean_df = pd.read_csv('./dirty_data/company_english.csv')


114

In [144]:
to_clean = [int(x[:3]) for x in os.listdir('.') if x.endswith('.json')]
to_clean.pop(to_clean.index(114))
for chunk_no in to_clean:
    chunk_websites = list(pd.read_csv('./targets/{}.csv'.format(chunk_no)).website.values)
    chunk_websites_set = set(chunk_websites)
    filename = '{}.json'.format(chunk_no)
    
    tmp = clean_df.set_index('website').loc[chunk_websites]
    with open(filename, 'r') as f:
        parsing_results = json.load(f)
    
    if chunk_no < 3:
        for x in parsing_results:
            x['original_url'] = deformat_link(x['url'])

    parsed_websites = {x['original_url'] for x in parsing_results}
    failed = [x for x in chunk_websites if x not in parsed_websites]
    pd.DataFrame(failed, columns=['website']).to_csv('./remaining_targets/{}.csv'.format(chunk_no), index=False)
    print(chunk_no, 'failed', len(failed))
    
    success = []
    saved = set()
    for x in parsing_results:
        if x['original_url'] not in chunk_websites_set or x['original_url'] in saved:
            continue
        row = tmp.loc[x['original_url']]
        x['country'] = row['country']
        x['region'] = row['region']
        x['locality'] = row['locality']
        x['founded'] = row['founded']
        x['industry'] = row['industry']
        x['size'] = row['size']
        if chunk_no > 2:
            x.pop('index')
        success.append(x)
        saved.add(x['original_url'])
    
    print(chunk_no, 'saved', len(success))
    with open('./clean_data/{}.json'.format(chunk_no), 'w') as f:
        json.dump(success, f)
    shutil.move(filename, './processed/'+filename)

In [146]:
total = 0
for f_name in os.listdir('./clean_data/'):
    with open('./clean_data/{}'.format(f_name), 'r') as f:
        l = json.load(f)
    total += len(l)
total

968685

In [118]:
targets = []
for f_name in os.listdir('./remaining_targets/'):
    targets += list(pd.read_csv('./remaining_targets/{}.csv'.format(f_name)).website.values)
len(targets)

2296602

In [120]:
targets = pd.DataFrame(targets, columns=['website'])
CHUNK_SIZE = 20000
n_chunks = ceil(len(targets)/CHUNK_SIZE)
for i in range(n_chunks):
    targets.iloc[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE].to_csv('./targets/{}.csv'.format(i+100), index=False)

In [123]:
{x['size'] for x in l}

{'1-10',
 '10001+',
 '1001-5000',
 '11-50',
 '201-500',
 '5001-10000',
 '501-1000',
 '51-200'}

In [128]:
clean_df['size'].unique()

array(['11-50', '1-10', '51-200', '1001-5000', '501-1000', '201-500',
       '10001+', '5001-10000'], dtype=object)

In [130]:
clean_df.head()

,Unnamed: 0,country,founded,id,industry,linkedin_url,locality,name,region,size,website,english
0,0,united kingdom,2017.0,codingcodex,education management,linkedin.com/company/codingcodex,nottingham,codex,nottingham,11-50,codingwithcodex.co.uk,True
1,4,united states,2007.0,cougar-electric-llc,construction,linkedin.com/company/cougar-electric-llc,swartz creek,cougar electric llc,michigan,1-10,power-now.biz,True
2,7,united states,NaN,miracle-doors-co-llp,utilities,linkedin.com/company/miracle-doors-co-llp,houston,"miracle doors, llc",texas,11-50,miracledoors.com,True
3,10,united states,1995.0,carlson-consulting-engineers,civil engineering,linkedin.com/company/carlson-consulting-engineers,memphis,"carlson consulting engineers, inc.",tennessee,11-50,carlsonconsulting.net,True
4,13,united states,2014.0,beacon-marketing-group-inc-by-owner-,marketing and advertising,linkedin.com/company/beacon-marketing-group-in...,kirkland,beacon marketing group inc,washington,1-10,beaconmarketinggroupinc.com,True


In [132]:
gr = clean_df.groupby('industry').count().sort_values('id', ascending=False)[['id', 'website']]

In [ ]:
gr